In [4]:
# Weights for objective function
alpha = 0.5
beta = 0.5
gamma = 0.5

warehouse_to_consider = ["FDW", "IDW", "PHR", "AW"]
using JuMP, HiGHS, GLPK, DataFrames, NamedArrays
using CSV, LinearAlgebra

# Reading the data as a DataFrame:
df = CSV.read("Data_For_Julia_50.csv", DataFrame; types=Dict(Symbol("Material Number") => String))

# For this instance we have 4 warehouses "McAllen - PHR, Florence - FDW, Indianapolis - IDW, Ontario - AW"
num_warehouse = length(warehouse_to_consider)             # As stated in the above comment
num_parts = nrow(df)          # Reads the number of rows in the DataFrame and number of parts.

part_name = df[:, 1]          # The first column in the DataFrame consists of "Manufacturing Material Numbers"

# The warehouse names are manually entered, this is used while displaying the results
#warehouse_name = ["AW", "FDW", "IDW", "PHR"]

# The DataFrame contains the part concentration data based on warehouses under the below mentioned column names
part_concentration = Matrix(df[:, ["% " * warehouse for warehouse in warehouse_to_consider]])

# The DataFrame contains the lead time data based on warehouses under the below mentioned column names
lead_time = Matrix(df[:, [warehouse * " Lead Time" for warehouse in warehouse_to_consider]])

# The DataFrame contains the ROP based on warehouses under the below mentioned column names
ROP = Matrix(df[:, [warehouse * " ROP" for warehouse in warehouse_to_consider]])

# Created NamedArrays for part concentration data
PCNA = NamedArray(part_concentration, (part_name, ["% " * warehouse for warehouse in warehouse_to_consider]), ("Part Number", "Warehouse Concentration"))

# Creating NamedArrays for the lead time data
LTNA = NamedArray(lead_time, (part_name, [warehouse * " Lead Time" for warehouse in warehouse_to_consider]), ("Part Number", "Lead Time"))

# Creating NamedArrays for the ROP
ROPNA = NamedArray(ROP, (part_name, [warehouse * " ROP" for warehouse in warehouse_to_consider]), ("Part Number", "ROP"))


# Model Initialization using HiGHS Solver
m1 = Model(GLPK.Optimizer)

# Decision Variable --> Declaring the variables to be binary
@variable(m1, x[1:num_parts, 1:num_warehouse], Bin)

#Constraint --> One part has to be assigned to one warehouse
@constraint(m1, [i in 1:num_parts], sum(x[i, j] for j in 1:num_warehouse) == 1)

# Objective function 1: Maximize the part concentration
@expression(m1, concentration_obj, sum(PCNA[i, j] * x[i, j] for i in 1:num_parts, j in 1:num_warehouse))

# Objective function 2: Minimize lead time
@expression(m1, lead_time_obj, sum(LTNA[i, j] * x[i, j] for i in 1:num_parts, j in 1:num_warehouse))

# Objective function 3: Minimize ROP
@expression(m1, ROP_obj, sum(ROPNA[i, j] * x[i, j] for i in 1:num_parts, j in 1:num_warehouse))

# Overall Weighted Objective function
@objective(m1, Max, alpha * concentration_obj - beta * lead_time_obj - gamma * ROP_obj)

#Constraint --> If any of the warehouse has customer concentration greater than equal to 80% then the part is assigned to this warehouse
for i in 1:num_parts
    for j in 1:num_warehouse
        if PCNA[i, j] >= 55
            @constraint(m1, x[i, j] == 1)
        end
    end
end
# Solve the model
optimize!(m1)

# Display the results
println("Objective value: ", objective_value(m1))

for i in 1:num_parts
    for j in 1:num_warehouse
        if value(x[i, j]) == 1
        #println("$(part_name[i]) in $(warehouse_name[j]): ", value(x[i, j]))
        println("$(part_name[i]) in $(warehouse_to_consider[j])")
        end
    end
end

Objective value: -15287.0
0006400000 in FDW
0006800000 in FDW
0008000000 in FDW
0008030000 in FDW
0008800000 in IDW
0008830000 in FDW
10039514 in PHR
10039515 in PHR
10039516 in PHR
10066377 in PHR
10066388 in PHR
10342890 in PHR
10350364 in PHR
10405801 in FDW
10405853 in IDW
10405888 in IDW
10405889 in IDW
10406012 in IDW
10406050 in IDW
10406074 in IDW
10406172 in FDW
10406312 in PHR
10406349 in IDW
10406476 in PHR
10406494 in IDW
10406496 in IDW
10406506 in IDW
10406507 in IDW
10406521 in IDW
10406671 in IDW
10406748 in IDW
10406850 in IDW
10406889 in IDW
10406900 in IDW
10406901 in IDW
10406934 in IDW
10406937 in IDW
10406942 in IDW
10406947 in IDW
10406988 in PHR
10407010 in IDW
10407013 in IDW
10407024 in PHR
10407033 in IDW
10407045 in IDW
10407139 in FDW
10407179 in PHR
10407221 in PHR
10407272 in IDW
10407306 in IDW
10407329 in IDW
10407342 in IDW
10407360 in PHR
10407369 in FDW
10407386 in PHR
10407555 in IDW
10407606 in IDW
10407609 in IDW
10407850 in IDW
10407853 in PHR
10

In [5]:

results = DataFrame(Part_Number = String[], Warehouse = String[])

for i in 1:num_parts
    for j in 1:num_warehouse
        if value(x[i, j]) ==1
            push!(results, (part_name[i], warehouse_to_consider[j]))
        end
    end
end

In [6]:
using CSV

CSV.write("Result_From_Julia_55.csv", results)

"Result_From_Julia_55.csv"